In [116]:
#Importando bibliotecas
import numpy as np
import pandas as pd

# Definindo a classe

In [117]:
class database:
    
    def __init__(self,pasta = ''):
        
        self.calendar = pd.read_csv(pasta+'calendar.csv')
        self.sales_train_evaluation = pd.read_csv(pasta+'sales_train_evaluation.csv')
        self.sales_train_validation = pd.read_csv(pasta+'sales_train_validation.csv')
        self.sample_submission = pd.read_csv(pasta+'sample_submission.csv')
        self.sell_prices = pd.read_csv(pasta+'sell_prices.csv')
        
        self.item_id =  self.sales_train_validation['item_id'].value_counts().index.sort_values()
        self.dept_id =  self.sales_train_validation['dept_id'].value_counts().index.sort_values()
        self.cat_id =  self.sales_train_validation['cat_id'].value_counts().index.sort_values()
        self.store_id = self.sales_train_validation['store_id'].value_counts().index.sort_values()
        self.state_id = self.sales_train_validation['state_id'].value_counts().index.sort_values()
        
        self.id =  self.sales_train_validation['id'].value_counts().index.sort_values()
        
        
        
    def get_labels(self, dataset = 'validation'):
        
        if dataset == 'evaluation':
            labels = self.sales_train_evaluation
            sales_id = self.sales_train_evaluation['id']
        elif dataset == 'validation':
            labels = self.sales_train_validation
            sales_id = self.sales_train_validation['id']
        labels = pd.DataFrame(labels.iloc[:,6:])
        labels = labels.transpose()
        labels.columns = sales_id
        
        self.eval_labels = labels
        
        return labels

    def get_item_features(self,item_id,store_id,days):
        sell_prices = self.sell_prices
        calendar = self.calendar
        
        sell_prices_update = sell_prices[sell_prices['store_id'] == store_id ][sell_prices['item_id'] == item_id ]
        calendar_update = calendar.iloc[:days,:]
        calendar_update = calendar_update[calendar_update['wm_yr_wk'].isin(sell_prices_update['wm_yr_wk'])]
        
        
        sell_prices_final = sell_prices_update
        for i in range(6):
            sell_prices_final = pd.concat([sell_prices_final, sell_prices_update])
            
        sell_prices_final = sell_prices_final.sort_values('wm_yr_wk')
        
        calendar_update = calendar_update.reset_index(drop = True)

        spf_size = len(sell_prices_final)
        cu_size = len(calendar_update)
        final_week_diff = 7 - calendar_update['wday'][cu_size-1]
        first_week_diff = calendar_update['wday'][0]-1
        
        
        sell_prices_final = sell_prices_final.iloc[first_week_diff:spf_size -final_week_diff,:]
        
        
        calendar_update['sell_price'] = sell_prices_final['sell_price'].reset_index(drop = True)
        
        return calendar_update
    
    
    
    # item full id example 'FOODS_1_001_CA_1_validation'
    def Hierarchy(self, cat='NaN', dept = 'NaN', store = 'NaN', state = 'NaN' ):
        
        hierarchycal_itens_list = []
        aux = 0
        
        
        item_ref = [cat, dept,"NaN", state, store,'NaN']
        
        for i in self.id:
            
            item = i.split('_')
            item[2] = 'NaN'
            item[5] = 'NaN'
            
            if cat == 'NaN':
                item[0] = 'NaN'
            if dept == 'NaN':
                item[1] = 'NaN'
            if state == 'NaN':
                item[3] = 'NaN'
            if store == 'NaN':
                item[4] = 'NaN'         
            
            if item_ref == item:
                hierarchycal_itens_list.append(i)
                
        return hierarchycal_itens_list

    def info(self):
        print("""
        ------------M5 Forecasting competition----------------
                        Informações da base
        
        """)

## Testando dados

In [118]:
pasta = 'Dados-originais/'
M5db = database(pasta = pasta)

In [119]:
M5db.calendar.head()

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1


In [120]:
M5db.sales_train_evaluation.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,4,0,0,0,0,3,3,0,1
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,1,2,1,1,0,0,0,0,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,2,0,0,0,2,3,0,1
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,1,0,4,0,1,3,0,2,6
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,2,1,0,0,2,1,0


In [121]:
M5db.sales_train_validation.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4


In [122]:
M5db.sell_prices.head()

,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.58
1,CA_1,HOBBIES_1_001,11326,9.58
2,CA_1,HOBBIES_1_001,11327,8.26
3,CA_1,HOBBIES_1_001,11328,8.26
4,CA_1,HOBBIES_1_001,11329,8.26


In [123]:
M5db.item_id

Index(['FOODS_1_001', 'FOODS_1_002', 'FOODS_1_003', 'FOODS_1_004',
       'FOODS_1_005', 'FOODS_1_006', 'FOODS_1_008', 'FOODS_1_009',
       'FOODS_1_010', 'FOODS_1_011',
       ...
       'HOUSEHOLD_2_507', 'HOUSEHOLD_2_508', 'HOUSEHOLD_2_509',
       'HOUSEHOLD_2_510', 'HOUSEHOLD_2_511', 'HOUSEHOLD_2_512',
       'HOUSEHOLD_2_513', 'HOUSEHOLD_2_514', 'HOUSEHOLD_2_515',
       'HOUSEHOLD_2_516'],
      dtype='object', length=3049)

In [124]:
M5db.dept_id

Index(['FOODS_1', 'FOODS_2', 'FOODS_3', 'HOBBIES_1', 'HOBBIES_2',
       'HOUSEHOLD_1', 'HOUSEHOLD_2'],
      dtype='object')

In [125]:
M5db.cat_id

Index(['FOODS', 'HOBBIES', 'HOUSEHOLD'], dtype='object')

In [126]:
M5db.store_id

Index(['CA_1', 'CA_2', 'CA_3', 'CA_4', 'TX_1', 'TX_2', 'TX_3', 'WI_1', 'WI_2',
       'WI_3'],
      dtype='object')

In [127]:
M5db.state_id

Index(['CA', 'TX', 'WI'], dtype='object')

# Testando Funções

## database.get_labels( dataset)
Retorna as labels do dataset desejado

In [128]:
M5db.get_labels(dataset = 'validation')

id,HOBBIES_1_001_CA_1_validation,HOBBIES_1_002_CA_1_validation,HOBBIES_1_003_CA_1_validation,HOBBIES_1_004_CA_1_validation,HOBBIES_1_005_CA_1_validation,HOBBIES_1_006_CA_1_validation,HOBBIES_1_007_CA_1_validation,HOBBIES_1_008_CA_1_validation,HOBBIES_1_009_CA_1_validation,HOBBIES_1_010_CA_1_validation,...,FOODS_3_818_WI_3_validation,FOODS_3_819_WI_3_validation,FOODS_3_820_WI_3_validation,FOODS_3_821_WI_3_validation,FOODS_3_822_WI_3_validation,FOODS_3_823_WI_3_validation,FOODS_3_824_WI_3_validation,FOODS_3_825_WI_3_validation,FOODS_3_826_WI_3_validation,FOODS_3_827_WI_3_validation
d_1,0,0,0,0,0,0,0,12,2,0,...,0,14,1,0,4,0,0,0,0,0
d_2,0,0,0,0,0,0,0,15,0,0,...,0,11,1,0,4,0,0,6,0,0
d_3,0,0,0,0,0,0,0,0,7,1,...,0,5,1,0,2,2,0,0,0,0
d_4,0,0,0,0,0,0,0,0,3,0,...,0,6,1,0,5,2,0,2,0,0
d_5,0,0,0,0,0,0,0,0,0,0,...,0,5,1,0,2,0,0,2,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
d_1909,1,1,1,0,1,0,1,4,0,0,...,4,1,1,0,0,0,0,1,1,0
d_1910,3,0,0,1,2,0,0,6,0,0,...,2,3,3,0,2,1,0,0,0,0
d_1911,0,0,1,3,2,2,0,3,0,2,...,0,1,6,0,3,0,0,0,3,0
d_1912,1,0,1,7,2,0,1,2,0,0,...,3,0,0,4,2,0,1,1,1,0


In [129]:
M5db.get_labels(dataset = 'evaluation')

id,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_006_CA_1_evaluation,HOBBIES_1_007_CA_1_evaluation,HOBBIES_1_008_CA_1_evaluation,HOBBIES_1_009_CA_1_evaluation,HOBBIES_1_010_CA_1_evaluation,...,FOODS_3_818_WI_3_evaluation,FOODS_3_819_WI_3_evaluation,FOODS_3_820_WI_3_evaluation,FOODS_3_821_WI_3_evaluation,FOODS_3_822_WI_3_evaluation,FOODS_3_823_WI_3_evaluation,FOODS_3_824_WI_3_evaluation,FOODS_3_825_WI_3_evaluation,FOODS_3_826_WI_3_evaluation,FOODS_3_827_WI_3_evaluation
d_1,0,0,0,0,0,0,0,12,2,0,...,0,14,1,0,4,0,0,0,0,0
d_2,0,0,0,0,0,0,0,15,0,0,...,0,11,1,0,4,0,0,6,0,0
d_3,0,0,0,0,0,0,0,0,7,1,...,0,5,1,0,2,2,0,0,0,0
d_4,0,0,0,0,0,0,0,0,3,0,...,0,6,1,0,5,2,0,2,0,0
d_5,0,0,0,0,0,0,0,0,0,0,...,0,5,1,0,2,0,0,2,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
d_1937,0,0,0,1,0,0,1,5,0,1,...,3,6,3,0,0,1,0,1,0,0
d_1938,3,0,2,3,0,0,0,4,0,1,...,1,4,3,1,2,0,1,0,1,2
d_1939,3,0,3,0,2,5,1,1,0,0,...,3,4,3,1,1,0,0,1,1,2
d_1940,0,0,0,2,1,2,1,40,1,0,...,0,1,0,0,3,1,1,0,1,5


## database.get_item_features( Item, Loja, Data )
Retorna todas features de um item de uma determinada loja até a data especificada

In [130]:
M5db.get_item_features('HOBBIES_1_001','CA_1',1913)

C:\Users\Julian\Anaconda3\lib\site-packages\ipykernel_launcher.py:41: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price
0,2013-07-13,11325,Saturday,1,7,2013,d_897,NaN,NaN,NaN,NaN,0,1,0,9.58
1,2013-07-14,11325,Sunday,2,7,2013,d_898,NaN,NaN,NaN,NaN,0,0,1,9.58
2,2013-07-15,11325,Monday,3,7,2013,d_899,NaN,NaN,NaN,NaN,0,1,1,9.58
3,2013-07-16,11325,Tuesday,4,7,2013,d_900,NaN,NaN,NaN,NaN,0,0,0,9.58
4,2013-07-17,11325,Wednesday,5,7,2013,d_901,NaN,NaN,NaN,NaN,0,0,0,9.58
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1012,2016-04-20,11612,Wednesday,5,4,2016,d_1909,NaN,NaN,NaN,NaN,0,0,0,8.38
1013,2016-04-21,11612,Thursday,6,4,2016,d_1910,NaN,NaN,NaN,NaN,0,0,0,8.38
1014,2016-04-22,11612,Friday,7,4,2016,d_1911,NaN,NaN,NaN,NaN,0,0,0,8.38
1015,2016-04-23,11613,Saturday,1,4,2016,d_1912,NaN,NaN,NaN,NaN,0,0,0,8.38


## database.Hierarchy( cat, dept, store, state )
Retorna todos ids de itens em uma ramificação da hierarquia especificada

In [131]:
M5db.Hierarchy(cat='HOBBIES', dept = '2', store = '2', state = 'TX' )

['HOBBIES_2_001_TX_2_validation',
 'HOBBIES_2_002_TX_2_validation',
 'HOBBIES_2_003_TX_2_validation',
 'HOBBIES_2_004_TX_2_validation',
 'HOBBIES_2_005_TX_2_validation',
 'HOBBIES_2_006_TX_2_validation',
 'HOBBIES_2_007_TX_2_validation',
 'HOBBIES_2_008_TX_2_validation',
 'HOBBIES_2_009_TX_2_validation',
 'HOBBIES_2_010_TX_2_validation',
 'HOBBIES_2_011_TX_2_validation',
 'HOBBIES_2_012_TX_2_validation',
 'HOBBIES_2_013_TX_2_validation',
 'HOBBIES_2_014_TX_2_validation',
 'HOBBIES_2_015_TX_2_validation',
 'HOBBIES_2_016_TX_2_validation',
 'HOBBIES_2_017_TX_2_validation',
 'HOBBIES_2_018_TX_2_validation',
 'HOBBIES_2_019_TX_2_validation',
 'HOBBIES_2_020_TX_2_validation',
 'HOBBIES_2_021_TX_2_validation',
 'HOBBIES_2_022_TX_2_validation',
 'HOBBIES_2_023_TX_2_validation',
 'HOBBIES_2_024_TX_2_validation',
 'HOBBIES_2_025_TX_2_validation',
 'HOBBIES_2_026_TX_2_validation',
 'HOBBIES_2_027_TX_2_validation',
 'HOBBIES_2_028_TX_2_validation',
 'HOBBIES_2_029_TX_2_validation',
 'HOBBIES_2_03

In [133]:
#Pegando todos itens de todas categorias só do estado do texas
M5db.Hierarchy(state = 'TX' )

['FOODS_1_001_TX_1_validation',
 'FOODS_1_001_TX_2_validation',
 'FOODS_1_001_TX_3_validation',
 'FOODS_1_002_TX_1_validation',
 'FOODS_1_002_TX_2_validation',
 'FOODS_1_002_TX_3_validation',
 'FOODS_1_003_TX_1_validation',
 'FOODS_1_003_TX_2_validation',
 'FOODS_1_003_TX_3_validation',
 'FOODS_1_004_TX_1_validation',
 'FOODS_1_004_TX_2_validation',
 'FOODS_1_004_TX_3_validation',
 'FOODS_1_005_TX_1_validation',
 'FOODS_1_005_TX_2_validation',
 'FOODS_1_005_TX_3_validation',
 'FOODS_1_006_TX_1_validation',
 'FOODS_1_006_TX_2_validation',
 'FOODS_1_006_TX_3_validation',
 'FOODS_1_008_TX_1_validation',
 'FOODS_1_008_TX_2_validation',
 'FOODS_1_008_TX_3_validation',
 'FOODS_1_009_TX_1_validation',
 'FOODS_1_009_TX_2_validation',
 'FOODS_1_009_TX_3_validation',
 'FOODS_1_010_TX_1_validation',
 'FOODS_1_010_TX_2_validation',
 'FOODS_1_010_TX_3_validation',
 'FOODS_1_011_TX_1_validation',
 'FOODS_1_011_TX_2_validation',
 'FOODS_1_011_TX_3_validation',
 'FOODS_1_012_TX_1_validation',
 'FOODS_